In [1]:
import random
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
path = '../data/raw/evrlearn/taxonomy_V4.csv'

In [3]:
taxonomy = pd.read_csv(path)

# remove all rows where the column 'ElementID' is null
taxonomy = taxonomy[taxonomy['ElementID'].notna()]

keep = ['ElementID', 'Dimension', 'Type Level 1', 'Type Level 1 E',
       'Type Level 2', 'Type Level 2 E', 'Type Level 3', 'Type Level 4']

# keep only the columns in the list 'keep'
taxonomy = taxonomy[keep]
names = ['Type Level 1', 'Type Level 2', 'Type Level 3', 'Type Level 4']

taxonomy['last_name'] = (taxonomy['ElementID'].str.len() - 1 )//2
taxonomy['last_name'] = taxonomy['last_name'].map(lambda x: names[x])
taxonomy['last_name'] = taxonomy.apply(lambda x: x[x['last_name']], axis=1)

In [121]:
taxonomy

,ElementID,Dimension,Type Level 1,Type Level 1 E,Type Level 2,Type Level 2 E,Type Level 3,Type Level 4,last_name
0,2.1,Deskriptor,Fertigkeit,Skill,Kognitive Fertigkeit,Cognitive Skills,NaN,NaN,Kognitive Fertigkeit
1,2.1.1,Deskriptor,Fertigkeit,Skill,Kognitive Fertigkeit,Cognitive Skills,Problemlösungsfähigkeit,NaN,Problemlösungsfähigkeit
2,2.1.1.1,Deskriptor,Fertigkeit,Skill,Kognitive Fertigkeit,Cognitive Skills,Problemlösungsfähigkeit,Problembewusstsein,Problembewusstsein
3,2.1.1.2,Deskriptor,Fertigkeit,Skill,Kognitive Fertigkeit,Cognitive Skills,Problemlösungsfähigkeit,Verarbeiten von Informationen,Verarbeiten von Informationen
4,2.1.1.3,Deskriptor,Fertigkeit,Skill,Kognitive Fertigkeit,Cognitive Skills,Problemlösungsfähigkeit,Analysieren von Daten oder Informationen,Analysieren von Daten oder Informationen
...,...,...,...,...,...,...,...,...,...
367,7.2.1,Kompetenz,Selbstkompetenz,Personal Competency,Selbstorganisation,Self-Organisation,Zeitmanagement,NaN,Zeitmanagement
368,7.2.2,Kompetenz,Selbstkompetenz,Personal Competency,Selbstorganisation,Self-Organisation,Emotionale Regulierung,NaN,Emotionale Regulierung
369,7.2.3,Kompetenz,Selbstkompetenz,Personal Competency,Selbstorganisation,Self-Organisation,Resilienz,NaN,Resilienz
370,7.2.4,Kompetenz,Selbstkompetenz,Personal Competency,Selbstorganisation,Self-Organisation,Ressourcenakquise,NaN,Ressourcenakquise


# Mastery Levels


In [4]:
mastery_levels = [1, 2, 3, 4]
nb_mastery_levels = len(mastery_levels)
mastery_levels_probabilities = [1/np.log(i+1) for i in range(1, nb_mastery_levels + 1)]
mastery_levels_normalized_probabilities = np.array(mastery_levels_probabilities) / sum(mastery_levels_probabilities)


# Skills

In [5]:
skills = taxonomy['last_name'].to_list()
random.shuffle(skills)
nb_skills = len(skills)
skills_probabilities = [1/np.log(i+1) for i in range(1, nb_skills + 1)]
skills_normalized_probabilities = np.array(skills_probabilities) / sum(skills_probabilities)

# Years

In [6]:
years = [i for i in range(2023, 2017, -1)]
years_probabilities = [1/np.log(i+1) for i in range(1, len(years) + 1)]
years_normalized_probabilities = np.array(years_probabilities) / sum(years_probabilities)

# Learners

In [7]:
def get_random_learner(skills, mastery_levels, years, min_n_skills=5, max_n_skills=10):
    n_skills = random.randint(min_n_skills, max_n_skills)
    possessed = {skill: level for skill, level in zip(np.random.choice(skills, n_skills, p=skills_normalized_probabilities, replace=False), np.random.choice(mastery_levels, n_skills, p=mastery_levels_normalized_probabilities, replace=True)) }
    year = np.random.choice(years, 1, p=years_normalized_probabilities)[0]
    return {'possessed_skills':possessed, 'year':year}

def get_all_learners(skills, mastery_levels, years, min_n_skills=5, max_n_skills=10, n_learners=100):
    return [get_random_learner(skills, mastery_levels, years, min_n_skills, max_n_skills) for _ in range(n_learners)]

learners = get_all_learners(skills, mastery_levels, years, min_n_skills=5, max_n_skills=10, n_learners=1000)

learners[0]

{'possessed_skills': {'Emotionale Regulierung': 1,
  'Kommunikation in Präsentations- und Vortragssituationen': 3,
  'Dynamische Kraft': 3,
  'Zusammenarbeit in stabilen Teams': 1,
  'Analysieren von Daten oder Informationen': 2},
 'year': 2023}

# Skill Supply

In [123]:
def get_skill_supply(learners, years):
    skill_supply = {year: Counter() for year in years}
    for learner in learners:
        for skill, level in learner['possessed_skills'].items():
            skill_supply[learner['year']][(skill, level)] += 1
    return skill_supply

skill_supply = get_skill_supply(learners, years)
skill_supply[2023].most_common(10)

[(('Räumliches Vorstellungsvermögen', 1), 31),
 (('Network Engineering', 1), 24),
 (('Räumliches Vorstellungsvermögen', 2), 21),
 (('Risikomanagement', 1), 17),
 (('Räumliches Vorstellungsvermögen', 3), 17),
 (('Qualitätsmanagement', 1), 15),
 (('Mitfühlen', 1), 14),
 (('Sprachliche Fertigkeiten', 1), 14),
 (('Motivieren', 1), 14),
 (('Reaktionszeit und Schnelligkeit', 1), 13)]

# Jobs

In [9]:
def get_random_job(skills, skills_normalized_probabilities, mastery_levels, mastery_levels_normalized_probabilities, years, years_normalized_probabilities, min_n_skills=2, max_n_skills=5):
    n_skills = random.randint(min_n_skills, max_n_skills)
    required = {skill: level for skill, level in zip(np.random.choice(skills, n_skills, p=skills_normalized_probabilities, replace=False), np.random.choice(mastery_levels, n_skills, p=mastery_levels_normalized_probabilities, replace=True)) }
    year = np.random.choice(years, 1, p=years_normalized_probabilities)[0]
    return {'required_skills':required, 'year':year}

def get_all_jobs(skills, skills_normalized_probabilities, mastery_levels, mastery_levels_normalized_probabilities, years, years_normalized_probabilities, min_n_skills=2, max_n_skills=5, n_jobs=1000):
    return [get_random_job(skills, skills_normalized_probabilities, mastery_levels, mastery_levels_normalized_probabilities, years, years_normalized_probabilities, min_n_skills, max_n_skills) for _ in range(n_jobs)]

jobs = get_all_jobs(skills, skills_normalized_probabilities, mastery_levels, mastery_levels_normalized_probabilities, years, years_normalized_probabilities, min_n_skills=2, max_n_skills=5, n_jobs=1000)

jobs[0]

{'required_skills': {'Verarbeiten von Informationen': 2,
  'Reaktionszeit und Schnelligkeit': 3,
  'Motivieren': 1,
  'Network Engineering': 1,
  'Risikomanagement': 3},
 'year': 2022}

# Skill Demand

In [10]:
def get_skill_demand(jobs, years):
    skill_demand = {year: Counter() for year in years}
    for job in jobs:
        for skill, level in job['required_skills'].items():
            skill_demand[job['year']][(skill, level)] += 1
    return skill_demand

skill_demand = get_skill_demand(jobs, years)
skill_demand[2023].most_common(10)

[(('Räumliches Vorstellungsvermögen', 1), 18),
 (('Network Engineering', 2), 11),
 (('Network Engineering', 1), 10),
 (('Network Engineering', 3), 9),
 (('Zusammenarbeit in stabilen Teams', 1), 8),
 (('Steuerungsfähigkeit', 1), 7),
 (('Analysieren von Daten oder Informationen', 1), 7),
 (('Qualitätsmanagement', 1), 7),
 (('Verhandeln', 1), 7),
 (('Zusammenarbeit in Projektteams', 1), 7)]

# Skill Attractiveness

In [11]:
def skill_attractiveness(skill, years, skill_supply, skill_demand):
    skill_attractiveness = 0
    noramlization_factor = 0
    for i, year in enumerate(years):
        if skill in skill_supply[year]:
            skill_attractiveness += skill_demand[year][skill] / (skill_supply[year][skill] * (i+1))
        noramlization_factor += 1 / (i+1)
    return skill_attractiveness/noramlization_factor

In [12]:
print(f"Skill demand in 2023: {skill_demand[2023][('Konzentrationsfähigkeit', 1)]}")
print(f"Skill supply in 2023: {skill_supply[2023][('Konzentrationsfähigkeit', 1)]}")
print(f"Skill demand in 2022: {skill_demand[2022][('Konzentrationsfähigkeit', 1)]}")
print(f"Skill supply in 2022: {skill_supply[2022][('Konzentrationsfähigkeit', 1)]}")
print(f"Skill attractiveness: {skill_attractiveness(('Konzentrationsfähigkeit', 1), years, skill_supply, skill_demand):.2f}")

Skill demand in 2023: 3
Skill supply in 2023: 5
Skill demand in 2022: 2
Skill supply in 2022: 3
Skill attractiveness: 0.52


# Learner - job Matching

In [120]:
def learner_job_matching(learner, job):
    matching = 0
    for skill in job['required_skills']:
        if skill in learner['possessed_skills']:
            sim = min(learner['possessed_skills'][skill], job['required_skills'][skill]) / job['required_skills'][skill]
            matching += sim
    matching = 100 * matching / len(job['required_skills'])
    return matching


nb_matching = 0
for learner, job in zip(learners, jobs):
    matching = learner_job_matching(learner, job)
    if matching > 30:
        print(f"Matching: {matching:.2f}")
        print(f"Number of skills in common: {len(set(learner['possessed_skills'].keys()).intersection(set(job['required_skills'].keys())))}")
        print(f"Job: {job['required_skills']}")
        print(f"Learner: {learner['possessed_skills']}")
        print()
        nb_matching += 1
    if nb_matching > 2:
        break

Matching: 37.50
Number of skills in common: 2
Job: {'Business- und Requirements-Engineerin': 3, 'Kommunikation in Präsentations- und Vortragssituationen': 3, 'Ergonomie, Usability': 2, 'Ambition': 1}
Learner: {'Service Engineering': 4, 'ICT Service Operation': 4, 'Gebäudeautomation': 2, 'Kommunikation in Präsentations- und Vortragssituationen': 3, 'Ergonomie, Usability': 1, 'Analysieren von Daten oder Informationen': 4, 'Führung als Empowerment': 2, 'Führung in der strategischen Ausrichtung': 1}

Matching: 66.67
Number of skills in common: 2
Job: {'Application Engineering': 1, 'Sprachliche Fertigkeiten': 1, 'Dynamische Kraft': 2}
Learner: {'Management der ICT': 3, 'Dynamische Kraft': 3, 'Genauigkeit': 1, 'Verarbeiten von Informationen': 2, 'Application Engineering': 1, 'Kommunikation in Feedbacksituationen': 1, 'Leistungsorientierung': 1, 'Problembewusstsein': 2, 'Physische Fertigkeit': 1}

Matching: 50.00
Number of skills in common: 1
Job: {'Motivieren': 4, 'Reaktionszeit und Schnelli

# Courses

In [44]:
def get_random_provided_skills(skills, mastery_levels, required_skills, n_provided_skills):
    provided_skills = dict()
    while len(provided_skills) < n_provided_skills:
        candidate_skill = random.choice(skills)
        candidate_level = random.choice(mastery_levels)
        if (
            candidate_skill not in required_skills
            and candidate_skill not in provided_skills
        ):
            provided_skills[candidate_skill] = candidate_level
        elif (
            candidate_skill in required_skills
            and candidate_level > required_skills[candidate_skill]
        ):
            provided_skills[candidate_skill] = candidate_level

    return provided_skills


def get_random_course(skills, mastery_levels, min_n_required_skills=1, max_n_required_skills=5, min_n_provided_skills=1, max_n_provided_skills=2):
    n_required_skills = random.randint(min_n_required_skills, max_n_required_skills)
    required = {skill: level for skill, level in zip(np.random.choice(skills, n_required_skills, replace=False), np.random.choice(mastery_levels, n_required_skills, replace=True)) }

    n_provided_skills = random.randint(min_n_provided_skills, max_n_provided_skills)
    provided = get_random_provided_skills(skills, mastery_levels, required, n_provided_skills)

    return {'required_skills':required, 'provided_skills':provided}

def get_all_courses(skills, mastery_levels, min_n_required_skills=1, max_n_required_skills=5, min_n_provided_skills=1, max_n_provided_skills=2, n_courses=1000):
    return [get_random_course(skills, mastery_levels, min_n_required_skills=1, max_n_required_skills=5, min_n_provided_skills=1, max_n_provided_skills=2) for _ in range(n_courses)]

courses = get_all_courses(skills, mastery_levels, min_n_required_skills=1, max_n_required_skills=5, min_n_provided_skills=1, max_n_provided_skills=2, n_courses=1000)

courses[0]

{'required_skills': {'Fokuswechsel': 2,
  'Psychomotorische Fertigkeit': 2,
  'Kommunikation in Feedbacksituationen': 4},
 'provided_skills': {'Kommunikation in Präsentations- und Vortragssituationen': 3,
  'Beurteilung der Qualitäten von Objekten, Dienstleistungen oder Personen': 2}}

# Learner-Course Matching

In [97]:
def learner_course_required_matching(learner, course):
    required_matching = 0
    for skill in course['required_skills']:
        if skill in learner['possessed_skills']:
            sim = min(learner['possessed_skills'][skill], course['required_skills'][skill]) / course['required_skills'][skill]
            required_matching += sim
    return required_matching / len(course['required_skills'])

def learner_course_provided_matching(learner, course):
    provided_matching = 0
    for skill in course['provided_skills']:
        if skill in learner['possessed_skills']:
            sim = min(learner['possessed_skills'][skill], course['provided_skills'][skill]) / course['provided_skills'][skill]
            provided_matching += sim
    return provided_matching / len(course['provided_skills'])

def learner_course_matching(learner, course):
    required_matching = learner_course_required_matching(learner, course)
    provided_matching = learner_course_provided_matching(learner, course)

    if provided_matching >= 1.0:
        return 0

    return required_matching / (provided_matching + 1)

In [98]:
nb_matching = 0
for learner, course in zip(learners, courses):
    matching = learner_course_matching(learner, course)
    if matching >= 0.1:
        print(f"Matching: {matching:.2f}")
        print(f"Course required: {course['required_skills']}")
        print(f"Course provided: {course['provided_skills']}")
        print(f"Learner: {learner['possessed_skills']}")
        nb_matching += 1
    if nb_matching > 5:
        break

Matching: 0.50
Course required: {'Tatkraft': 4, 'Ergonomie, Usability': 1}
Course provided: {'Architekturen': 4}
Learner: {'Risikomanagement': 4, 'Führung in der strategischen Ausrichtung': 2, 'Unabhängigkeit': 4, 'Zusammenarbeit in virtuellen Teams': 4, 'Gleichgewicht': 2, 'Mitfühlen': 3, 'Raumwahrnehmung und -vorstellung': 3, 'Ergonomie, Usability': 1, 'Führung in Veränderungssituationen': 1, 'Räumliches Vorstellungsvermögen': 4}
Matching: 0.10
Course required: {'Risikomanagement': 4, 'Vergleichsfähigkeit': 2, 'Reaktionszeit und Schnelligkeit': 2, 'Führung in Konfliktsituationen': 2, 'ICT Service Operation': 4}
Course provided: {'Anleiten und Begleiten': 3}
Learner: {'Feinmotorik': 1, 'Durchhaltevermögen': 2, 'Beurteilung der Qualitäten von Objekten, Dienstleistungen oder Personen': 3, 'Originalität ': 3, 'Führung in Konfliktsituationen': 1, 'Führung in der strategischen Ausrichtung': 1, 'Kognitive Fertigkeit': 2, 'Informationssicherheit': 4, 'Teamorientierung': 2, 'Aufmerksamkeit': 

# Learner-skill Achievability

In [113]:
def learner_skill_achievability(learner, skill, mastery_level, courses):
    achievability = 0
    nb_courses = 0
    for course in courses:
        if skill in course['provided_skills'] and mastery_level == course['provided_skills'][skill]:
            achievability += learner_course_required_matching(learner, course)
            nb_courses += 1
    return achievability, nb_courses

In [114]:
learners[0]

{'possessed_skills': {'Emotionale Regulierung': 1,
  'Kommunikation in Präsentations- und Vortragssituationen': 3,
  'Dynamische Kraft': 3,
  'Zusammenarbeit in stabilen Teams': 1,
  'Analysieren von Daten oder Informationen': 2},
 'year': 2023}

In [115]:
jobs[0]

{'required_skills': {'Verarbeiten von Informationen': 2,
  'Reaktionszeit und Schnelligkeit': 3,
  'Motivieren': 1,
  'Network Engineering': 1,
  'Risikomanagement': 3},
 'year': 2022}

In [117]:
nb_achievable = 0

for learner in learners: 
    achievability, nb_courses = learner_skill_achievability(learner, 'Motivieren', 1, courses)
    if achievability > 0:
        nb_achievable += 1
        print(f"Achievability: {achievability:.2f}")
        print(f"Number of courses: {nb_courses}")
        print(f"Learner: {learner['possessed_skills']}")
        print(f"Skill: Motivieren")
        if nb_achievable > 5:
            break

Achievability: 0.25
Number of courses: 3
Learner: {'Beweglichkeit, Gleichgewicht und Koordination': 3, 'Dehnbarkeit': 1, 'Aufmerksamkeit': 1, 'Schriftlicher Ausdrucksfähigkeit': 1, 'Körperkoordination': 1, 'Network Engineering': 2, 'Numerische Fertigkeiten': 2, 'Mustererkennung': 2, 'Architekturen': 2}
Skill: Motivieren
Achievability: 0.33
Number of courses: 3
Learner: {'Problemlösungsfähigkeit': 2, 'Führung in der strategischen Ausrichtung': 1, 'Zeitmanagement': 3, 'Merkfähigkeit': 2, 'Service Design': 3, 'Zusammenarbeit in Projektteams': 1, 'Selbstbeherrschung': 2}
Skill: Motivieren
Achievability: 0.62
Number of courses: 3
Learner: {'Führung in Entwicklungssituationen': 1, 'Mündliche Ausdrucksfähigkeit': 3, 'Führung in Veränderungssituationen': 1, 'Aufnahmefähigkeit': 1, 'Durchhaltevermögen': 3, 'Gebäudeautomation': 4, 'Technical Software Engineering': 1, 'Statische Kraft': 1, 'Resilienz': 3, 'Beweglichkeit, Gleichgewicht und Koordination': 2}
Skill: Motivieren
Achievability: 1.00
Nu